In [1]:
import numpy as np
import pandas as pd
import tweepy
%matplotlib inline
from matplotlib import pyplot as plt
import requests
import os 
import timeit
import time
import json

# Gathering Data

- [x] Download twitter_archive_enhanced.csv
- [x] Download image_predictions.tsv
- [x] Download tweet_json.txt

## tasks
> Download file from [link ](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv) using requests
> save the file into the root derectory using os.
>
>> check whether the file exists.
>
> Download file from [link](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)
>> check whether the file exists.

> Query Twitter API for each tweet's JSON data.

>> 1. store each tweet's entire set of JSON data in a file called tweet_json.txt.

>> 2. each tweet's JSON data should be written to its own line.

>> 3. Then read this .txt file line by line into a pandas DataFrame with tweet ID, retweet count and favourite count.

In [2]:
def gather_data(url, file_name):
    if os.path.exists(file_name):
        return "Already exist"
    else:
        r = requests.get(url)
        with open(file_name, 'wb') as file:
            file.write(r.content)
        return 'downloaded'
twitter_archive_enhanced_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv'
gather_data(twitter_archive_enhanced_url, 'twitter-archive-enhanced.csv')

'Already exist'

In [3]:
image_predictions_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
gather_data(image_predictions_url, 'image_predictions.tsv')

'Already exist'

In [4]:
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')

In [5]:
image_predictions = pd.read_csv('image_predictions.tsv', delimiter='\t')

In [6]:
tweet_id = '851953902622658560'

### Try the API extract json

### Count the retweet and like of every tweet id.

### store every tweet id's json as one entity in json.txt


>> 1. Extract ids from tweet data file and prediction file.
        df.id, tweepy.API.get_status
>> 2. save the into tweet_json.txt 
        json.dump

### test 

In [7]:
# read tweet_archive_file and image_prediction file
tweet_archive_f = pd.read_csv('twitter-archive-enhanced.csv')
prediction_f = pd.read_csv('image_predictions.tsv', delimiter='\t')

In [8]:
# check if the tsv file read succefully
prediction_f.sample()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1345,759159934323924993,https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg,1,Irish_terrier,0.254856,True,briard,0.227716,True,soft-coated_wheaten_terrier,0.223263,True


In [9]:
# collect ids from above two tables.
tweet_ids = pd.Series(list(tweet_archive_f.tweet_id) + list(prediction_f.tweet_id))

In [10]:
tweet_ids.sample(2)

4287    859074603037188101
75      878281511006478336
dtype: int64

In [11]:
len(tweet_ids)

4431

There is 4431 ids.

In [12]:
tweet_ids.count()

4431

None of them are null

In [13]:
consumer_key = 'secret'
consumer_secret = 'secret'
access_token = 'secret'
access_secret = 'secret'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True) # Set rate limit to avoid errors.

def download_tweets_json(tweetid):
    '''Download tweet using tweet's ids json file.
    
    key arguments
    tweetid -- the tweetid which you wanna download.
    '''
    # start_time = time.process_time()
    #if os.path.exists('json.txt'):
    #    return "json.txt exists"
    try:
        status = api.get_status(tweetid, tweet_mode='extended') # 获取id的所有状态信息.
        return json.dumps(status._json) # 把status自带的json转换为json第三方包可读的json.类型是str.
    except tweepy.TweepError as e:
        print(tweet_id+'disappeared')
        pass
    # print('%d used'%(time.process_time() - start_time))
    print('tweet ID: %s'%(tweetid)) # Print ID after required

# Assess Data

Assess there data sets and note at least 8 quality problems and 2 tideness problem.

1. you only want ratings(no retweets) that have images. Not all the tweets are about dogs and some are retweets.
2. 8 quality issues and 2 tideness issues in the dataset.
3. Merging individual pieces of data according to the rules of tidy data.
4. Numertators are greater than the denominators.(this is normal)
5. Do not gather the tweets beyond August 1st, 2017.

# 数据整洁性问题
1. 数据完整性.
2. 数据可用性.
3. 数据合理性.
4. 数据一致性.

## 表1: twitter_archive_enhanced

In [14]:
twitter_archive_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### 完整性问题(变量空值问题)
1. in_reply_to_status_id
2. in_reply_to_user_id
3. retweeted_stated_id
4. retweeted_status_user_id
5. expanded_urls

### 数据可用性问题
1. id类,name,text应该是字符串
2. source需要把网址提取出来,iPhone,Twitter,vine,tweetdeck.
3. 小狗种类应该是布林值.

## 数据整洁性问题
1. 每一列是一个变量.
2. 每一行是一个样本.
3. 每一个表是一个观察角度.

### 每一列是一个变量.
1. 转发的三列包含了两个意思, 第一个是是否转发, 第二个才是转发的具体信息, 所以应该有一列是布林值的retweet.
2. 同理,回复列的in_reply_to的两个变量也是, id是value,而列的名字应该是变量名, 所以有一列应该是布林值的replay.
3. 狗狗的成长阶段是value,应该作为stage的值,所以应该新建一个stage的列,把值填进去.

### 每一个表是一个观察角度.
1. 回复类的和转发类的应该是两个单独的表.

## 表2: image_predictions

In [15]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### 质量问题
1. tweet_id 应该是string.
2. jpg_url 应该是string.

# 表3 json.txt

 read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. Note: do not include your Twitter API keys, secrets, and tokens in your project submission.

### BUG
1. 我把每一个id读出来的json放在一行, 但是现在的问题是, json已经无法识别这个json文件.现在变成了一个string.

思路: 重新写Twitter API的获取json文件的代码.每一行保存成json object而不是str, 或者利用json的库来实现将json转换为str.

In [16]:
keys = ['id', 'retweet_count', 'favorite_count']

### Gather data from tweeter API
需求: 我现在需要做的是, 写出一个函数, 输入json.txt, 输出是一个dataframe, 变量为 id, retweet_count, favourite_couint.

define
> 使用df完成所有的操作.

1. 读取json.txt文件为Series.
2. 对于每一条读取为json object.
3. 对于每个json object提取需要的key放入字典

In [17]:
# 尝试步骤
json_original = pd.read_csv('json.txt', header=None)

json_original.sample(5)

json.loads(json_original[0][0])['id']

dict(zip(keys, [1, 2, 3]))

one_row = pd.DataFrame(dict(zip(keys, [1, 2, 3])), index=[0])

one_row.append(one_row)

[json.loads(json_original[0][0])[key] for key in keys]

pd.DataFrame(columns=keys)

,id,retweet_count,favorite_count


思路:
1. 读取文件.
2. 用json.loads把每一行解析成字典.
3. 利用key选出需要的键值.
4. 利用append把每一行添加进dataframe.

In [18]:
# 所有包含的keys
json.loads(json_original[0][0]).keys()

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'extended_entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'possibly_sensitive_appealable', 'lang'])

In [19]:
def create_df_from_json(file_name, keys):
    """
    返回一个拥有特定keys的dataframe
    -----
    参数: 
    -----
    file_name: 字符串, 文件地址
    keys: 需要提取的键值
    """
    json_original = pd.read_csv(file_name, header=None) # 读取json文件
    df = pd.DataFrame(columns=keys) # 创建只含有表头的新dataframe, 
    # 提取keys并作为一行dataframe
    index = 0
    for json_object in json_original[0]:
        try:
            json_dict = json.loads(json_object) # 读取成json的dictionary.
        except TypeError:
            print(index, "failes") # 有些空值的文件读取不了就算了.
        make_dict = dict(zip(keys, [json_dict[key] for key in keys]))
        df = df.append(pd.DataFrame(make_dict, index=[index]))
        index += 1
    return df

# Assessing json.txt data

In [20]:
json_df = create_df_from_json('json.txt', keys)

19 failes
95 failes
118 failes
132 failes
155 failes
247 failes
260 failes
298 failes
382 failes
566 failes
784 failes
3971 failes
4191 failes
4220 failes
4300 failes
4348 failes
4411 failes


In [21]:
json_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4431 entries, 0 to 4430
Data columns (total 3 columns):
favorite_count    4431 non-null object
id                4431 non-null object
retweet_count     4431 non-null object
dtypes: object(3)
memory usage: 138.5+ KB


In [22]:
json_df.describe()

,favorite_count,id,retweet_count
count,4431,4431,4431
unique,2135,2345,1748
top,0,888078434458587136,307
freq,245,3,8


## Quality issues
1. id应该是string, 其余的应该是正整数.
2. Twitter id unique的只有2345个, 说明有大量重复id.

### 查看重复id的条目信息, 看看是不是重复数据

In [23]:
json_df[json_df.duplicated()]

,favorite_count,id,retweet_count
19,20013,888554962724278272,3643
95,34924,874012996292530176,10693
118,37170,870063196459192321,8614
132,32858,867051520902168576,8188
155,76674,862096992088072192,24475
247,26692,845677943972139009,5229
260,22916,843235543001513987,6653
298,17181,837110210464448512,2659
382,17195,827324948884643840,3423
566,6953,802265048156610565,1542


In [24]:
json_df[json_df.id == 888554962724278272]

,favorite_count,id,retweet_count
18,20013,888554962724278272,3643
19,20013,888554962724278272,3643
4412,20013,888554962724278272,3643


是重复的

### tideness issues
1. 重复数据.
2. 表1,表2,表3可以合并.

# Clean data
## define
### quality issues
表1
1. in_reply_to_status_id
2. in_reply_to_user_id
3. retweeted_stated_id
4. retweeted_status_user_id
5. expanded_urls
6. id类,name,text应该是字符串
7. source需要把网址提取出来,iPhone,Twitter,vine,tweetdeck.
8. 小狗种类应该是布林值.

表2
1. tweet_id 应该是string.
2. jpg_url 应该是string.

### tideness issues
表1
1. 转发的三列包含了两个意思, 第一个是是否转发, 第二个才是转发的具体信息, 所以应该有一列是布林值的retweet.
2. 同理,回复列的in_reply_to的两个变量也是, id是value,而列的名字应该是变量名, 所以有一列应该是布林值的replay.
3. 狗狗的成长阶段是value,应该作为stage的值,所以应该新建一个stage的列,把值填进去.
4. 回复类的和转发类的应该是两个单独的表.

表3
1. 重复数据.
2. 表1,表2,表3可以合并.

## code

In [282]:
# 解决表1的空值问题.
twitter_archive_enhanced_clean = twitter_archive_enhanced.copy()

In [283]:
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

@TODO: 对于这几个空值我该如何清理?
1. in_reply相关: 这是一条回复用的tweet. 不一定需要删除, 空值说明是原创的tweet.

从上面的表可以看出来,回复用的tweet是没有狗狗图片的,所以应该删除.

In [284]:
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean[twitter_archive_enhanced_clean.in_reply_to_status_id.isnull()]

In [285]:
# 删除掉这两列
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean.drop(['in_reply_to_status_id','in_reply_to_user_id'],axis=1)

In [286]:
# test 看看删除之后的效果
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id                      2278 non-null int64
timestamp                     2278 non-null object
source                        2278 non-null object
text                          2278 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2274 non-null object
rating_numerator              2278 non-null int64
rating_denominator            2278 non-null int64
name                          2278 non-null object
doggo                         2278 non-null object
floofer                       2278 non-null object
pupper                        2278 non-null object
puppo                         2278 non-null object
dtypes: float64(2), int64(3), object(10)
memory usage: 284.8+ KB


define: 删除掉转发的Twitter
@TODO: 如何判断是不是转发的?
retweeted的意思: 里面是原tweeter的id,利用retweeted_status,non_null说明这是一条转发的, 所以应该删除.

In [287]:
# code 
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean[twitter_archive_enhanced_clean.retweeted_status_id.isnull()]
# 删除转发的三列
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1)

In [288]:
# test
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
source                2097 non-null object
text                  2097 non-null object
expanded_urls         2094 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: int64(3), object(9)
memory usage: 213.0+ KB


## define
删除没有关系的列.

In [289]:
uselessColumns = ['expanded_urls', 'source']

In [290]:
# 删除没用的列
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean.drop(uselessColumns, axis=1)

In [291]:
# test
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null object
text                  2097 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: int64(3), object(7)
memory usage: 180.2+ KB


### define: 把数据的类型变为正确的
1. tweet_id, text, name, doggo, floofer, pupper, puppo是string
2. timestamp 是时间

In [292]:
#code
twitter_archive_enhanced_clean.timestamp = pd.to_datetime(twitter_archive_enhanced_clean.timestamp)

In [293]:
#test
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id              2097 non-null int64
timestamp             2097 non-null datetime64[ns]
text                  2097 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 180.2+ KB


In [294]:
# change the others into string.
twitter_archive_enhanced_clean.tweet_id = twitter_archive_enhanced_clean.tweet_id.apply(str)

In [295]:
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 10 columns):
tweet_id              2097 non-null object
timestamp             2097 non-null datetime64[ns]
text                  2097 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
doggo                 2097 non-null object
floofer               2097 non-null object
pupper                2097 non-null object
puppo                 2097 non-null object
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 180.2+ KB


# define: 小狗的stage作为一列

In [296]:
# code
twitter_archive_enhanced_clean.columns

Index(['tweet_id', 'timestamp', 'text', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [297]:
for stage in dogStages:
    print(twitter_archive_enhanced_clean[stage].unique())

['None' 'doggo']
['None' 'floofer']
['None' 'pupper']
['None' 'puppo']


说明每一列都是None或者狗的stage

In [298]:
def stage_name(df):
    for stageName in df[dogStages]:
        if stageName != 'None':
            return stageName
    return None

In [299]:
twitter_archive_enhanced_clean['stage'] = twitter_archive_enhanced_clean.apply(stage_name, axis=1)

In [300]:
# delete dog specific stages
twitter_archive_enhanced_clean = twitter_archive_enhanced_clean.drop(dogStages, axis=1)

In [301]:
# test
twitter_archive_enhanced_clean.sample(10)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,stage
845,766423258543644672,2016-08-18 23:55:18,This is Shadoe. Her tongue flies out of her mo...,9,10,Shadoe,None
100,872820683541237760,2017-06-08 14:20:41,Here are my favorite #dogsatpollingstations \n...,13,10,None,None
410,823322678127919110,2017-01-23 00:13:17,This is Wyatt. He's got the fastest paws in th...,11,10,Wyatt,None
2,891815181378084864,2017-07-31 00:18:03,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,None
1200,716730379797970944,2016-04-03 20:53:33,There has clearly been a mistake. Pup did noth...,12,10,None,None
975,750041628174217216,2016-07-04 19:00:33,This is Beau. He's trying to keep his daddy fr...,13,10,Beau,None
1237,712309440758808576,2016-03-22 16:06:19,Reminder that we made our first set of sticker...,12,10,None,pupper
674,789599242079838210,2016-10-21 22:48:24,This is Brownie. She's wearing a Halloween the...,12,10,Brownie,None
15,889278841981685760,2017-07-24 00:19:32,This is Oliver. You're witnessing one of his m...,13,10,Oliver,None
1700,680959110691590145,2015-12-27 03:51:18,This is Ozzie. He was doing fine until he lost...,9,10,Ozzie,None


In [302]:
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 7 columns):
tweet_id              2097 non-null object
timestamp             2097 non-null datetime64[ns]
text                  2097 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
stage                 336 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 131.1+ KB


In [303]:
twitter_archive_enhanced_clean['point'] = twitter_archive_enhanced_clean.rating_numerator / twitter_archive_enhanced_clean.rating_denominator

In [304]:
twitter_archive_enhanced_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id              2097 non-null object
timestamp             2097 non-null datetime64[ns]
text                  2097 non-null object
rating_numerator      2097 non-null int64
rating_denominator    2097 non-null int64
name                  2097 non-null object
stage                 336 non-null object
point                 2097 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 147.4+ KB


## 表2 clean data

In [305]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [306]:
image_predictions_clean = image_predictions.copy()

In [307]:
image_predictions_clean.tweet_id = image_predictions_clean.tweet_id.apply(str)

In [308]:
image_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


In [309]:
# combine image_prediction and twitter_archive
df_clean = image_predictions_clean.merge(twitter_archive_enhanced_clean, on='tweet_id')

In [310]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 1970
Data columns (total 19 columns):
tweet_id              1971 non-null object
jpg_url               1971 non-null object
img_num               1971 non-null int64
p1                    1971 non-null object
p1_conf               1971 non-null float64
p1_dog                1971 non-null bool
p2                    1971 non-null object
p2_conf               1971 non-null float64
p2_dog                1971 non-null bool
p3                    1971 non-null object
p3_conf               1971 non-null float64
p3_dog                1971 non-null bool
timestamp             1971 non-null datetime64[ns]
text                  1971 non-null object
rating_numerator      1971 non-null int64
rating_denominator    1971 non-null int64
name                  1971 non-null object
stage                 303 non-null object
point                 1971 non-null float64
dtypes: bool(3), datetime64[ns](1), float64(4), int64(3), object(8)
memory

## clean 表3

In [311]:
json_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4431 entries, 0 to 4430
Data columns (total 3 columns):
favorite_count    4431 non-null object
id                4431 non-null object
retweet_count     4431 non-null object
dtypes: object(3)
memory usage: 138.5+ KB


In [312]:
json_clean = json_df.copy()

In [313]:
json_clean = json_clean.rename(columns={'id': 'tweet_id'})

In [314]:
json_clean.tweet_id = json_clean.tweet_id.apply(str)

In [315]:
json_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4431 entries, 0 to 4430
Data columns (total 3 columns):
favorite_count    4431 non-null object
tweet_id          4431 non-null object
retweet_count     4431 non-null object
dtypes: object(3)
memory usage: 138.5+ KB


In [316]:
# drop duplicate ones in json_clean
json_clean = json_clean[json_clean.tweet_id.duplicated()]

In [317]:
# merge the three dataframe
df_clean = df_clean.merge(json_clean, on='tweet_id')

In [319]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1986 entries, 0 to 1985
Data columns (total 21 columns):
tweet_id              1986 non-null object
jpg_url               1986 non-null object
img_num               1986 non-null int64
p1                    1986 non-null object
p1_conf               1986 non-null float64
p1_dog                1986 non-null bool
p2                    1986 non-null object
p2_conf               1986 non-null float64
p2_dog                1986 non-null bool
p3                    1986 non-null object
p3_conf               1986 non-null float64
p3_dog                1986 non-null bool
timestamp             1986 non-null datetime64[ns]
text                  1986 non-null object
rating_numerator      1986 non-null int64
rating_denominator    1986 non-null int64
name                  1986 non-null object
stage                 306 non-null object
point                 1986 non-null float64
favorite_count        1986 non-null object
retweet_count         1986 non-

In [320]:
df_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,...,p3_dog,timestamp,text,rating_numerator,rating_denominator,name,stage,point,favorite_count,retweet_count
752,688916208532455424,https://pbs.twimg.com/media/CY-Fn1FWEAQhzhs.jpg,1,Pembroke,0.430544,True,red_fox,0.206576,False,Pomeranian,...,True,2016-01-18 02:49:58,This pupper just wants to say hello. 11/10 wou...,11,10,None,pupper,1.1,2936,960
1019,712065007010385924,https://pbs.twimg.com/media/CeHDV73W0AM5Cf8.jpg,1,goose,0.214301,False,gibbon,0.084253,False,pizza,...,False,2016-03-21 23:55:01,This is Chester. He's clearly in charge of the...,6,10,Chester,None,0.6,2419,691
492,675853064436391936,https://pbs.twimg.com/media/CWEcxqWVEAAHyGH.jpg,1,Labrador_retriever,0.868367,True,golden_retriever,0.043305,True,vizsla,...,True,2015-12-13 01:41:41,Here we have an entire platoon of puppers. Tot...,88,80,None,None,1.1,2852,1417
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,...,True,2015-11-15 23:21:54,Here is a very happy pup. Big fan of well-main...,9,10,a,None,0.9,125,45
554,678255464182861824,https://pbs.twimg.com/media/CWmlvxJU4AEAqaN.jpg,1,Chihuahua,0.613819,True,Yorkshire_terrier,0.127931,True,Pomeranian,...,True,2015-12-19 16:47:58,This is Jessifer. She is a Bismoth Teriyaki. F...,9,10,Jessifer,None,0.9,1693,408


In [322]:
df_clean.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'timestamp', 'text',
       'rating_numerator', 'rating_denominator', 'name', 'stage', 'point',
       'favorite_count', 'retweet_count'],
      dtype='object')

In [327]:
# delete useless columns
df_clean = df_clean.drop(['p2','p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog','rating_numerator', 'rating_denominator', 'img_num'], axis=1)

In [328]:
# test
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1986 entries, 0 to 1985
Data columns (total 12 columns):
tweet_id          1986 non-null object
jpg_url           1986 non-null object
p1                1986 non-null object
p1_conf           1986 non-null float64
p1_dog            1986 non-null bool
timestamp         1986 non-null datetime64[ns]
text              1986 non-null object
name              1986 non-null object
stage             306 non-null object
point             1986 non-null float64
favorite_count    1986 non-null object
retweet_count     1986 non-null object
dtypes: bool(1), datetime64[ns](1), float64(2), object(8)
memory usage: 268.1+ KB


In [333]:
# drop duplicated ones
df_clean = df_clean.drop_duplicates('tweet_id')

In [334]:
# save df_clean
df_clean.to_csv('twitter_archive_master.csv')